In [1]:
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b
import csv
import numpy as np
import math
from collections import defaultdict
import operator
import hashlib
import copy

In [2]:
with open('categories.txt', 'rb') as content_file:
    categories = content_file.read().split('\n')
#to make things faster i put the index where every category starts in a dictionary where the key are the product ids and 
#the values are the starting index of the category
catindex = defaultdict(int)
f = open('categories.txt','rb')
for i,l in enumerate(f):
    if len(l)-len(l.strip()) <=1:
        catindex[l.strip()]=i
        
#takes product id and returns category
def getCategory(pid, categories, catdict, depth):
    index = catdict[pid]
    value = set()
    j = index + 1
    while len(categories[j])-len(categories[j].strip())>1:
                ws = (categories[j].strip()).split(',')
                break
    return str((ws[:depth])[-1].strip())

In [3]:
data = []
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], float(row['rating']), row['time']))

In [4]:
data[0], len(data)

(('A1YXHALQFWN45C', '0783885008', 4.0, '1288310400'), 67046)

In [5]:
def gethexdig(txt):
    c = hashlib.md5(txt.encode())
    return c.hexdigest()

gs= defaultdict(int)
for d in data:
    cat = getCategory(d[1], categories, catindex, 3)
    gs[cat]+=1
gs = set(sorted(gs.iteritems(), key= operator.itemgetter(1), reverse=True)[:50])
ghashed = set()
for g in gs:
    ghashed.add(gethexdig(g[0])) 

In [6]:
itemsuserrated = defaultdict(lambda: list())
useritems = defaultdict(lambda: list())
itemsuserratedwithtime = defaultdict(lambda: list())
for d in data:
    itemsuserrated[d[0]].append((d[1], d[3]))

for u in itemsuserrated:
    items = itemsuserrated[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    itemsuserrated[u] = isorted
    useritems[u] = [i[0] for i in isorted]
    itemsuserratedwithtime[u] = [(i[0], i[1]) for i in isorted]

In [7]:
datag = []
depth = 3
count = 0
dg = 'anything'
for d in data:
    index = list(useritems[d[0]]).index(d[1])
    cgenre = getCategory(d[1],categories,catindex,depth)
    if gethexdig(cgenre) not in ghashed:
        cgenre = dg
    if not index==0:
        pgenre = getCategory(list(useritems[d[0]])[index-1], categories, catindex, depth)
        if gethexdig(pgenre) not in ghashed:
            pgenre = dg
        tup = (d[:3] + (cgenre, pgenre))
    else: 
        tup = (d[:3] + (cgenre,))
    datag.append(tup)


In [8]:
datag[0]

('A1YXHALQFWN45C', '0783885008', 4.0, 'Historical', 'Food & Wine')

In [9]:
users = {}
products = {}
us = set()
ps = set()
genres = defaultdict(int)
gs  = set()
i = 0
j = 0
k = 0
for d in datag:
    if d[0] not in us:
        users[d[0]] = i
        i+=1
        us.add(d[0])
    if d[1] not in ps:
        products[d[1]] = j
        j+=1
        ps.add(d[1])
    if d[3] not in gs:
        gs.add(d[3])
        genres[d[3]] = k
        k+=1

In [10]:
len(itemsuserrated), len(users), len(genres)

(357, 357, 51)

In [53]:
# # datag = list(set(datag))
# # np.random.shuffle(datag)
# dtest = []
# dt = set()
# dv = set()
# dvalid= [] #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for d in datag:
#     cr = len(itemsuserrated[d[0]])
#     vc = int((cr * 20)/float(100))
#     if dtestusers[d[0]]< vc:
#         dtestusers[d[0]] +=1
#         dt.add(d)
#         dtest.append(d)
#     if d not in dt and dvalidusers[d[0]]<vc:
#         dvalidusers[d[0]]+=1
#         dv.add(d)
#         dvalid.append(d)

# dtrain = []
# for d in datag:
#     if d not in dt and d not in dv:
#         dtrain.append(d)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [82]:
x60 = int((len(datag) * 60) / float(100))
x20 = int((len(datag) * 20) / float(100))
dtrain = datag[:x60]
dvalid = datag[x60: x60+ x20]
end =  x60+ x20
dtest = datag[end:]
print len(dtrain), len(dvalid), len(dtest), len(datag)
print len(dtrain) + len(dvalid) + len(dtest) == len(datag)

40227 13409 13410 67046
True


In [83]:
dtrain[0]

('A1YXHALQFWN45C', '0783885008', 4.0, 'Historical', 'Food & Wine')

In [84]:
print len(users), len(products)
len(dtrain)+ len(dtest)+ len(dvalid) == len(datag)

357 50123


True

In [85]:
gside = len(genres)
gside

51

In [86]:
# k = 1
# lam = 10
# s = 0
# for d in datag:
#     s+= d[2]
# alpha = s / float(len(data))
# betau = np.random.normal(0,0.1, len(users))
# betai = np.random.normal(0,0.1, len(products))
# betagg = np.random.normal(0,0.1, (gside, gside))
# initial = [alpha] + list(betau) + list(betai) + list(betagg.flatten())
# gammu = np.random.normal(0,0.1, (len(users), k)) 
# gammi = np.random.normal(0,0.1, (len(products), k)) 
# initial= initial + list(gammu.flatten()) + list(gammi.flatten())
# compute_cost(initial, 0, 1, dtrain)

In [87]:
us = set()
ps = set()
gs = set()
for d in datag:
    us.add(d[0])
    ps.add(d[1])
    for i in xrange(4, len(d)):
        gs.add((d[i], d[3]))

In [88]:
uintrain = set()
pintrain = set()
gintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    uintrain.add(d[0])
    for i in xrange(4, len(d)):
        gintrain.add((d[i], d[3]))

In [89]:
print len(gs), len(gintrain)

2322 2077


In [90]:
def getPredictions(params,k, data):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    bij = np.array(params[end: end+(gside**2)])
    betaggo = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(params[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    predictions = []
    for d in data:
        pred = alphao + betauo[users[d[0]]] + betaio[products[d[1]]] + (gammuo[users[d[0]]].T.dot(gammio[products[d[1]]]))
        for i in xrange(4, len(d)):
            pred += betaggo[genres[d[i]]][genres[d[3]]]
        predictions.append(pred)
    return predictions

In [91]:
def mse(params,k, data):
    s = 0
    predictions = getPredictions(params, k, data)
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return s / float(len(data))

In [92]:
def mean(data):
    s = 0
    for d in data:
        s+= int(d[2])
    return  s/ float(len(data))

In [93]:
mean(data)

4.161471228708648

In [94]:
def variance(data):
    m = mean(data)
    s = 0
    for d in data:
        s += (m - d[2]) ** 2
    return s/float(len(data))

In [95]:
def Rsquare_statistic(params,k, data):
    predictions = getPredictions(params,k, data)
    error = mse(params,k, data)
    var = variance(data)
    return error/float(var)

In [96]:
def rmse(params,k,data):
    predictions = getPredictions(params, k, data)
    s = 0
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return math.sqrt(s / float(len(data)))

In [97]:
def compute_cost(initial,lam1, lam2,k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betagg = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        term = (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2])
        for i in xrange(4, len(d)):
            term += betagg[genres[d[i]]][genres[d[3]]]
        J+= (term ** 2)
    for u in us:
        J += lam1 * (np.square(betau[users[u]])+ np.sum(np.square(gammu[users[u]])))
    for p in ps:
        J+= lam1 * (np.square(betai[products[p]]) + np.sum(np.square(gammi[products[p]])))
    for g in gintrain:
        J+= lam2 * np.square(betagg[genres[g[0]]][genres[g[1]]])
    g = compute_gradient(alpha, betau, betai, betagg, gammu, gammi,lam1, lam2,k, data)
#     print J
    return J,g.flatten()

In [98]:
def cost(initial,k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betagg = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        term = (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2])
        for i in xrange(4, len(d)):
            term += betagg[genres[d[i]]][genres[d[3]]]
        J+= (term ** 2)

    return J

In [99]:
def compute_gradient(alpha,betau, betai,betagg, gammu, gammi,lam1, lam2,k, data):
    ga = 0
    gbu = np.zeros(len(users))
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    gbgg = np.zeros((gside, gside))
    for d in data:
        term = (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2])
        for i in xrange(4, len(d)):
            term += betagg[genres[d[i]]][genres[d[3]]]
        ga+= 2 * term
        gbu[users[d[0]]] += 2 * term
        gbi[products[d[1]]] += 2 * term
        ggu[users[d[0]]] += 2 * gammi[products[d[1]]] * term
        ggi[products[d[1]]] += 2 * gammu[users[d[0]]] * term
        for i in xrange(4, len(d)):
            gbgg[genres[d[i]]][genres[d[3]]] += 2* term
    for u in us:
        gbu[users[u]] += 2 * lam1 * betau[users[u]]
        ggu[users[u]] += 2 * lam1 * gammu[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam1 * betai[products[p]]
        ggi[products[p]] += 2 * lam1 * gammi[products[p]]
    for g in gs:
        gbgg[genres[g[0]]][genres[g[1]]] += 2 * lam2 * betagg[genres[g[0]]][genres[g[1]]]
    return np.array([ga] + list(gbu) + list(gbi) + list(gbgg.flatten()) + list(ggu.flatten())+ list(ggi.flatten()))

In [100]:
k = 10
s = 0
for d in data:
    s+= d[2]
alpha = s / float(len(data))
betau = np.random.normal(0,0.1, len(users))
betai = np.random.normal(0,0.1, len(products))
betagg = np.random.normal(0,0.1, (gside, gside))
gammu = np.random.normal(0,0.1, (len(users), k)) 
gammi = np.random.normal(0,0.1, (len(products), k)) 
for u in users:
    if u not in uintrain:
        betau[users[u]] = 0
        gammu[users[u]] = 0

for p in products:
    if p not in pintrain:
        betai[products[p]] = 0
        gammi[products[p]] = 0

for g in gs:
    if g not in gintrain:
        betagg[genres[g[0]]][genres[g[1]]] = 0

initial = [alpha] + list(betau) + list(betai) + list(betagg.flatten()) + list(gammu.flatten()) + list(gammi.flatten())

In [101]:
# class ConvergedException(Exception):
#     pass
# def check(xk):
#     global result
#     global minmse
#     global count
#     global presult
#     global nit 
    
#     nit +=1
# #     print nit

#     cmse = mse(xk,k, dvalid)
    
#     if cmse > minmse:
#         if nit >= 100:
#             count +=1
#             if count == 5:
#                 print 'here'
#                 result[:] = presult
#                 raise ConvergedException()
#                 return

#     else:
#         minmse = cmse
#         presult[:] = xk
#         count = 0
# #     print cmse, minmse, presult[0]
#     return
    

In [102]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

#     cmse = mse(xk,k, dvalid)
    cmse = cost(xk, k, dvalid)
    if cmse > minmse:
            count +=1
            if count == 30:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [103]:
lams1 = [10]
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
for l1 in lams1:
    for l2 in lams2:
        minmse = 1000000000000000
        presult = []
        count = 0
        nit = 0
        result = []
        params = []
        try:
            params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1, l2,k, dtrain, ), disp = 0, callback = check_updated)
            params[:] = presult
        except ConvergedException:
            params[:] = result
        print 'converged' + str(nit)
        print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" : ' + ' ' + str(cost(params, k, dvalid)) + ' ' + str(cost(params,k, dtest)) + ' ' +  str(mse(params, k, dvalid)) + ' ' + str(mse(params,k, dtest))

here
converged61
lambda "10" -> "0" :  10488.337303 10275.8192303 0.782186389961 0.76628033037
here
converged61
lambda "10" -> "0.0001" :  10488.3348074 10275.8163691 0.782186203845 0.766280117009
here
converged61
lambda "10" -> "0.001" :  10488.3123511 10275.7906198 0.78218452913 0.766278196854
here
converged61
lambda "10" -> "0.01" :  10488.0882234 10275.5332556 0.78216781441 0.766259004891
here
converged61
lambda "10" -> "0.1" :  10485.8896221 10272.976665 0.782003849811 0.766068356822
here
converged61
lambda "10" -> "1" :  10465.6257189 10249.8994118 0.780492633221 0.764347458001
here
converged65
lambda "10" -> "10" :  10377.8202943 10148.4927598 0.77394438767 0.756785440702
here
converged94
lambda "10" -> "100" :  10314.3545837 10103.9027631 0.769211319538 0.753460310445
here
converged78
lambda "10" -> "1000" :  10329.0007385 10113.2821639 0.770303582553 0.754159743768
here
converged113
lambda "10" -> "10000" :  10334.2942251 10110.5908403 0.770698353726 0.753959048493
here
conver

In [40]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
x = lams
y = [0.782691640296 , 0.78269300387  , 0.782707711788  , 0.782389661747  , 0.783028716481 , 0.781518211899, 0.77358537001,
     0.768996194757 , 0.769847381673, 0.770736449983 , 0.80777145789 
    ]
plt.scatter(lams[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')
plt.xlabel('Lambda Value For Adjacent Genre Bias Terms')
plt.ylabel('mse on Validation Set')
plt.title("Adjacent Pairwise Genre Bias Terms Model Regularization Parameter vs. mse")
plt.show()

In [90]:
lams1 = [10]
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
for l1 in lams1:
    for l2 in lams2:
        minmse = 1000000000000000
        presult = []
        count = 0
        nit = 0
        result = []
        params = []
        try:
            params, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1, l2,k, dtrain, ), disp = 0, callback = check_updated)
            params[:] = presult
        except ConvergedException:
            params[:] = result
        print 'converged' + str(nit)
        print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" : ' + ' ' + str(mse(params, k, dvalid)) + ', ' + str(mse(params, k, dtest))

here
converged73
lambda "10" -> "0" :  0.782663790376, 0.765423822909
here
converged73
lambda "10" -> "0.0001" :  0.78266369207, 0.765425618876
here
converged73
lambda "10" -> "0.001" :  0.78266313332, 0.765442903214
here
converged72
lambda "10" -> "0.01" :  0.782672024304, 0.765625225481
here
converged73
lambda "10" -> "0.1" :  0.782482471001, 0.765606061292
here
converged73
lambda "10" -> "1" :  0.7815432061, 0.764626843147
here
converged110
lambda "10" -> "10" :  0.773519620686, 0.756809869323
here
converged91
lambda "10" -> "100" :  0.768832602376, 0.753051600261
here
converged136
lambda "10" -> "1000" :  0.76979610544, 0.752800736982
here
converged114
lambda "10" -> "10000" :  0.771090234186, 0.754196350854
here
converged43
lambda "10" -> "100000" :  0.80152495266, 0.787062305622


In [42]:
try:
    pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(10,1000,k, dtrain, ), disp = 0, callback = check_updated)
except ConvergedException:
    pms = result
print mse(pms, k, dtest)

here
0.753883940781


In [116]:
lams1 = [10]
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
# with open('Regression baseline - Adjacent pair wise genre bias terms 100 core more.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda1', 'lambda2','k','value'])
    writer.writeheader()
    for l1 in lams1:
        for l2 in lams2: 
            minmse = 10000
            presult = []
            count = 0
            nit = 0
            result = []
            params = []
            try:
                params, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l1, l2,k, dtrain, ), disp = 0, callback = check)
                params[:] = presult
            except ConvergedException:
                params[:] = result
            print 'converged' + str(nit)
            print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" : ' + ' ' + str(mse(params, k, dtest))
            for i, v in enumerate(params):
                writer.writerow({'lambda1': l1, 'lambda2': l2,  'k': k, 'value': v})  

here
converged104
lambda "10" -> "0" :  0.766153915529
here
converged104
lambda "10" -> "0.0001" :  0.766260425001
here
converged104
lambda "10" -> "0.001" :  0.766658491669
here
converged104
lambda "10" -> "0.01" :  0.766684247062
here
converged104
lambda "10" -> "0.1" :  0.766381137135
here
converged104
lambda "10" -> "1" :  0.76430908397
here
converged104
lambda "10" -> "10" :  0.756883829114
here
converged108
lambda "10" -> "100" :  0.753114112668
here
converged104
lambda "10" -> "1000" :  0.753286957465
here
converged104
lambda "10" -> "10000" :  0.754408602709
here
converged108
lambda "10" -> "100000" :  0.754930544152


In [117]:
params = defaultdict(lambda: defaultdict(list))
with open('Regression baseline - Adjacent pair wise genre bias terms 100 core more.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        params[(float(row['lambda1']), float(row['lambda2']))][int(row['k'])].append(float(row['value']))

In [118]:
op = list()
minCost = 1000000000
predictions = []
for l in params:
    for k in params[l]:
        c = mse(params[l][k], k, dvalid)
#         print l, k, c
        print l, mse(params[l][k], k, dtest)
        if c < minCost:
            minCost = c
            minlambda = l
            mink = k
            op[:] = params[l][k]
mink, minlambda, minCost

(10.0, 1000.0) 0.753286957465
(10.0, 0.01) 0.766684247062
(10.0, 0.001) 0.766658491669
(10.0, 100.0) 0.753114112668
(10.0, 10.0) 0.756883829114
(10.0, 10000.0) 0.754408602709
(10.0, 0.1) 0.766381137135
(10.0, 0.0001) 0.766260425001
(10.0, 1.0) 0.76430908397
(10.0, 100000.0) 0.754930544152
(10.0, 0.0) 0.766153915529


(10, (10.0, 100.0), 0.76892429610416935)

In [122]:
# with open("Adjacent Model Optimized Parameters.csv", "wb") as f:
#     writer = csv.writer(f)
#     for o in op:
#         writer.writerow([o])

In [123]:
op = []
with open("Adjacent Model Optimized Parameters.csv", 'rb') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        op.append(float(row[0]))

In [124]:
print "mse: " + str(mse(op,k,dtest))
print "R Squared Statistic: " + str( mse(op, k, dtest) /float(variance(data)))

mse: 0.753114112668
R Squared Statistic: 0.76665385043


In [121]:
mse(params[(10,0)][10], k, dtrain)

0.58041664199319487

In [53]:
usergenres = defaultdict(lambda: set())
for u in useritems:
    for i in useritems[u]:
        cat = getCategory(i, categories, catindex, 3)
        if gethexdig(cat) not in ghashed:
            cat = 'Anything'
        usergenres[u].add(cat)

In [43]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]

x = lams2
y = [0.777347003886, 0.777518819439, 0.777791524404, 0.776386939524, 0.776732703502, 0.776313685638, 0.770003419702, 0.76692445357,
    0.767260668956, 0.767691347886, 0.807221417837]
# color = ['blue'] * 7 + ['red'] + ['blue'] * 3
plt.scatter(lams2[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')
# legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.xlabel('Lambda Value For Adjacent Genre Bias Terms')
plt.ylabel('mse on Validation Set')
plt.title("Adjacent Pairwise Genre Bias Terms Model Regularization Graph")
plt.show()
from matplotlib.backends.backend_pdf import PdfPages


In [36]:
# params, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(0,0,10, dtrain, ), disp = 0)
# mse(params,k, dtrain), mse(params,k, dvalid), mse(params,k, dtest)

In [135]:
params = op
alphao = params[0]
betauo = params[1: len(users)+1]
end = len(users)+1
betaio = params[end: end+len(products)]
end = end+len(products)
bij = np.array(params[end: end+(gside**2)])
betaggo = bij.reshape(gside, gside)
end = end + (gside**2)
gs = np.array(params[end:])
gr = gs.reshape(len(users)+len(products),k)
gammuo = gr[:len(users),:]
gammio = gr[len(users):,:]

In [136]:
data[0]

('A1G37DFO8MQW0M', 'B000BSF4ES', 5.0, '1311984000')

In [39]:
product_names = defaultdict(str)
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        product_names[row['pid']] = row['ptitle']

In [40]:
user_ratings_with_time = defaultdict(lambda: list())
user_ratings = defaultdict(lambda: list())
for d in data:
    user_ratings_with_time[d[0]].append((d[1], int(d[3])))

for u in user_ratings_with_time:
    items = user_ratings_with_time[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    user_ratings_with_time[u] = isorted
    user_ratings[u] = [i[0] for i in isorted]

In [41]:
#returns both items user has rated but not yet at this point in time and items not rated at all 
def get_products_not_rated(user, time):
    items = user_ratings_with_time[user]
    not_rated_yet = []
    for i,p in enumerate(items):
        if p[1] > int(time) and i > 0:
            not_rated_yet.extend([j[0] for j in items[i:]])
            break
    return ps.difference(user_ratings[user]).union(set(not_rated_yet))

In [42]:
def get_items_rated_in_future(user, time, pnames):
    items = user_ratings_with_time[user]
    not_rated_yet = []
    for i,p in enumerate(items):
        if p[1] > int(time) and i > 0:
            not_rated_yet.extend([j[0] for j in items[i:]])
            break
    names = []
    for n in not_rated_yet:
        names.append(pnames[n])
    return names

In [43]:
# get_items_rated_in_future(user, time, product_names)

In [44]:
def get_item_last_rated(user, time):
    items = user_ratings_with_time[user]
    for i,p in enumerate(items):
        if p[1] >= int(time) and i > 0:
            return getCategory(items[i-1][0], categories,catindex, 3)
    return ""


In [44]:
#This user read harry potter at this unix time
user = 'ABMX8XUNPR3LP'
time = '1265587200'


In [45]:
get_item_last_rated(user,time)

'Literature & Fiction'

In [46]:
inr = list(get_products_not_rated(user,time))
print inr[0]

007141858X


In [47]:
import operator
def getTopNRecommendations(user,time, n, pnames, params, k):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    bij = np.array(params[end: end+(gside**2)])
    betaggo = bij.reshape(gside, gside)
    end = end + (gside**2)
    gs = np.array(params[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    rankings = defaultdict(float)
    inr = get_products_not_rated(user,time)
    pcat = get_item_last_rated(user,time)
    for r in inr:
        ccat = getCategory(r, categories, catindex, 3)
        pred = alphao + betauo[users[user]] + betaio[products[r]] + (gammuo[users[user]].T.dot(gammio[products[r]]))
        if pcat!='':
            pred += betaggo[genres[pcat]][genres[ccat]]
        rankings[r] = pred
    rs = sorted(rankings.iteritems(), key = operator.itemgetter(1), reverse= True)[:n]
    names = []
    for r in rs:
#         if pnames[r[0]] == '':
#             print r[0]
        names.append(pnames[r[0]])
    return names
    

In [49]:
recs = getTopNRecommendations(user,time, 30, product_names, op, mink)
# for r in recs:
#     print str(r) + '\n'

In [50]:
# for p in user_ratings[user]:
#     print getCategory(p, categories,catindex, 3)

In [52]:
count = 0
nyr = get_items_rated_in_future(user, time, product_names)
for r in recs:
    if r in nyr:
        print r
        count+=1
if count == len(nyr):
    print True

World War Z: An Oral History of the Zombie War
The Giver


In [53]:
product_names['007141858X']

'How to Talk to Anyone: 92 Little Tricks for Big Success in Relationships'

In [54]:
# getTopNRecommendations_genres(user,time, 20, op, mink)


In [75]:
c = 0
pgi = defaultdict(float)
for p in products:
    cat = getCategory(p, categories,catindex,2)
#     if gethexdig(cat) not in ghashed:
#         cat = dg
#     pgi[product_names[p]] = gammio[products[p]][4]
    pgi[cat] = gammio[products[p]][4]
#     c+=1
#     if c == 20:
#         break

In [76]:
pgiSorted = dict(sorted(pgi.iteritems(), key = operator.itemgetter(1), reverse=True)[:20])

In [77]:
pgiSorted

{'Arts & Photography': 0.0,
 'Biographies & Memoirs': 0.00066797175782883013,
 'Books': 0.0013545588271336941,
 "Children's Books": 0.011659103027814083,
 'Christian Books & Bibles': 0.0,
 'Cookbooks': 0.0,
 'Law': 0.006977030877938232,
 'Literature & Fiction': 0.024345373331861892,
 'Makeup': 0.0013938394865582507,
 'Medical Books': 0.0,
 'Mystery': 0.0,
 'New': -0.00036024999434565799,
 'Politics & Social Sciences': 0.0019258073819532679,
 'Professional & Technical': 0.0,
 'Religion & Spirituality': 0.0010990861845958876,
 'Science & Math': 0.043528131343161497,
 'Science Fiction & Fantasy': 0.0017927081948150771,
 'Self-Help': 0.0,
 'Teens': 0.015424787999956566,
 'Travel': 0.0}

In [78]:
# for i, p in enumerate(pgiSorted[:10]):
#     print p[0][0], ' : ', p[0][1], ' - ', p[0][2],   '\n'

In [79]:
import pandas as pd
frame = pd.DataFrame(data=pgiSorted,index= range(1), dtype=float)
frame = frame.fillna(value=0)

import seaborn as sns
import matplotlib.pyplot as plt

mask = np.zeros_like(frame, dtype=np.bool)
for i in range(frame.shape[0]):
    for j in range(frame.shape[1]):
        if frame.columns[j] == frame.index[i]:
            mask[i,j] = True

with sns.axes_style('whitegrid'):
    ax = sns.heatmap(frame, mask=mask,center= 0,  robust=True,linewidths=0.5)
plt.yticks(rotation=0) 
plt.xticks(rotation=90)
fig = plt.gcf()
plt.title('Adjacent Migrations - depth: '+ str(depth))
fig.subplots_adjust(bottom=0.3, left = 0.2)
plt.show()

In [155]:
def computeNumericalGradient(params,grad, lam,k, data):
    e = 1e-4
    numgrads = np.zeros(len(params))
    pp = np.zeros(len(params))
    for i,p in enumerate(params):
        pp[i] = e
        loss1,_ = compute_cost(params-pp, lam, k,data)
        loss2,_ = compute_cost(params+pp, lam,k, data)
#         print loss1, loss2
        numgrads[i] = (loss2 - loss1) / float(2*e)
        pp[i] = 0
        print round(numgrads[i], 3), round(grad[i], 3)
#         print abs(numgrads[i]-grad[i])
    return numgrads

In [137]:
def get_genre_from_id(i):
    for g in genres:
        if genres[g] == i:
            return g

In [138]:
ggbias = defaultdict(int)
ggbiasd = defaultdict(lambda: defaultdict(int))
for i in range(len(betaggo)):
    for j in range(len(betaggo[i])):
        g1 = get_genre_from_id(i)
        g2 = get_genre_from_id(j)
        ggbias[(g1,g2)] = betaggo[i][j]
        ggbiasd[g1][g2] = betaggo[i][j]

In [139]:
# sorted(ggbias.iteritems(), key=operator.itemgetter(1), reverse=True)

In [140]:
import pandas as pd
frame = pd.DataFrame.from_dict(data=ggbiasd, dtype=float)
frame = frame.fillna(value=0)

import seaborn as sns
import matplotlib.pyplot as plt

mask = np.zeros_like(frame, dtype=np.bool)
for i in range(frame.shape[0]):
    for j in range(frame.shape[1]):
        if frame.columns[j] == frame.index[i]:
            mask[i,j] = True

with sns.axes_style('whitegrid'):
    ax = sns.heatmap(frame, mask=mask,center= 0,  robust=True,linewidths=0.5)
plt.yticks(rotation=0) 
plt.xticks(rotation=90)
fig = plt.gcf()
plt.title('Adjacent Migrations - depth: '+ str(depth))
fig.subplots_adjust(bottom=0.3, left = 0.2)
plt.show()

In [342]:
fig.savefig("From Model Adjacent Migrations.pdf")

In [28]:
# params = op
# alphao = params[0]
# betauo = params[1: len(users)+1]
# end = len(users)+1
# betaio = params[end: end+len(products)]
# end = end+len(products)
# bij = np.array(params[end: end+(gside**2)])
# betaggo = bij.reshape(gside, gside)
# end = end + (gside**2)
# gs = np.array(params[end:])
# gr = gs.reshape(len(users)+len(products),k)
# gammuo = gr[:len(users),:]
# gammio = gr[len(users):,:]

In [47]:
def gradient(initial,lam,k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betagg = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    ga = 0
    gbu = np.zeros(len(users))
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    gbgg = np.zeros((gside, gside))
    for d in data:
        term = (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2])
        for i in xrange(4, len(d)):
            term += betagg[genres[d[i]]][genres[d[3]]]
        ga+= 2 * term
        gbu[users[d[0]]] += 2 * term
        gbi[products[d[1]]] += 2 * term
        ggu[users[d[0]]] += 2 * gammi[products[d[1]]] * term
        ggi[products[d[1]]] += 2 * gammu[users[d[0]]] * term
        for i in xrange(4, len(d)):
            gbgg[genres[d[i]]][genres[d[3]]] += 2* term
    for u in us:
        gbu[users[u]] += 2 * lam * betau[users[u]]
        ggu[users[u]] += 2 * lam * gammu[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam * betai[products[p]]
        ggi[products[p]] += 2 * lam * gammi[products[p]]
#     for g in gs:
#         gbgg[genres[g[0]]][genres[g[1]]] += 2 * 1000 * betagg[genres[g[0]]][genres[g[1]]]
    return np.array([ga] + list(gbu) + list(gbi) + list(gbgg.flatten()) + list(ggu.flatten())+ list(ggi.flatten()))

In [54]:
def computeNumericalGradient(params,grad, lam,k, data):
    e = 1e-4
    numgrads = np.zeros(len(params))
    pp = np.zeros(len(params))
    for i,p in enumerate(params):
        pp[i] = e
        loss1, _ = compute_cost(params-pp, lam, k,data)
        loss2, _ = compute_cost(params+pp, lam,k, data)
#         print loss1, loss2
        numgrads[i] = (loss2 - loss1) / float(2*e)
        pp[i] = 0
        print round(numgrads[i], 3), round(grad[i], 3)
#         print abs(numgrads[i]-grad[i])
    return numgrads

In [55]:
grad = gradient(initialg, 1,k , dtest[:1])
computeNumericalGradient(initialg, grad, 1,k, dtest[:1] )

-1.461 -1.461
-0.417 -0.417
1.246 1.246
-0.48 -0.48
-1.568 -1.568
0.135 0.135
-0.099 -0.099
-1.146 -1.146
1.452 1.452
-0.051 -0.051
-0.784 -0.784
0.115 0.115
0.0 0.0
0.0 0.0
-0.151 -0.151
0.103 0.103
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
-0.012 -0.012
0.137 0.137
0.276 0.276
0.025 0.025
0.429 0.429
-0.077 -0.077
-0.037 -0.037
0.0 0.0
0.0 0.0
0.0 0.0
-0.058 -0.058
-0.173 -0.173
0.0 0.0
0.246 0.246
-0.369 -0.369
-0.142 -0.142
-0.085 -0.085
-0.08 -0.08
0.0 0.0
0.0 0.0
0.287 0.287
0.0 0.0
0.2 0.2
-0.077 -0.077
0.0 0.0
0.0 0.0
0.0 0.0
0.029 0.029
0.0 0.0
-0.368 -0.368
0.042 0.042
-0.188 -0.188
-0.215 -0.215
0.0 0.0
-0.127 -0.127
-0.052 -0.052
0.151 0.151
-0.151 -0.151
-0.082 -0.082
0.0 0.0
0.0 0.0
0.102 0.102
0.0 0.0
0.0 0.0
0.272 0.272
-0.247 -0.247
0.0 0.0
-0.576 -0.576
0.0 0.0
0.286 0.286
-0.377 -0.377
-0.1 -0.1
0.123 0.123
0.0 0.0
0.17 0.17
0.055 0.055
0.0 0.0
-0.075 -0.075
0.0 0.0
-0.303 -0.303
0.03 0.03
0.0 0.0
-0.019 -0.019
-0.353 -0.353
-0.214 -0.214
0.0 0.0
0.185 0.185
0.294 0.294
0.0 0.

KeyboardInterrupt: 